# Get xT Values with and without pressure

Calculation of xT extends the model presented by https://alpscode.com/blog/expected_threat_optimization/ . Our model considers pressure as it has a crucial influence on an actions outcome.

In [1]:
import sasoptpy as so
import pandas as pd
import os
from mplsoccer import Pitch, FontManager, Sbopen
from math import sqrt
import random
import math
from mip import Model, MAXIMIZE, CBC, INTEGER, OptimizationStatus
import pulp
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from tqdm import tqdm


## Data Retrieval <a class='anchor' id='retrieval'></a>
We retrieve all available data from women's soccer from statsbomb.

In the following functions, we define what constitutes a success for a carry, because the Outcome-label of the data is not sufficient as it is rarely filled. Thus we are often looking at the following event to check if a carry was successful. For example, if a player completes a pass after a carry, the carry is evaluated as successful.

Note: The definitions for success and failure are tailor-made for carries and cannot simply be transferred to other event data like passes. However, we need to include all event data in the first place order to obtain an appropriate temporal granularity.

In [2]:
def get_event_data(only_female = True):
    #Get all availabe competitions

    df_competitions = pd.read_excel('data/available_competitions.xlsx')
    if only_female:
        df_competitions = df_competitions[df_competitions["competition_gender"]=="female"]
    df_competitions = df_competitions[df_competitions['available_matches'] > 4]
    match_ids = []
    parser = Sbopen(dataframe=True)
    for index, row in df_competitions.iterrows():
        df_matches = parser.match(competition_id=row['competition_id'], season_id = row['season_id'])
        match_ids.extend(df_matches.match_id.unique())
    print(len(match_ids))
    all_events_df = []
    all_frames_df = []
    all_visible_df = []
    ### filter kick off
    set_pieces = ['Throw-in', 'Free Kick', 'Goal Kick', 'Corner', 'Kick Off', 'Penalty']
    cols = ['match_id', 'id', "period", 'minute', 'second', 'type_name', 'sub_type_name', 'player_name',
            'x', 'y', 'end_x', 'end_y', 'outcome_name','team_name','under_pressure','position_name']
    for match_id in tqdm(match_ids):
        event = parser.event(match_id)[0]  # get the first dataframe (events)
        event = event.loc[(~event['sub_type_name'].isin(set_pieces))][cols].copy()# remove set-piece event
        try: # avoiding JSON parsing errors
            frame, visible = parser.frame(match_id)
            all_frames_df.append(frame)
            all_visible_df.append(visible)
        except:
            pass
        all_events_df.append(event)

    events = pd.concat(all_events_df).drop_duplicates()
    frames = pd.concat(all_frames_df).drop_duplicates()
    visibles = pd.concat(all_visible_df)
    events = events.reset_index()
    events = get_outcome_from_events(events)

    # every statsbomb dribble is related to a carry, so we merge those related events to one
    dribble_events = events[events["type_name"]=="Dribble"].copy()
    events = events[events["type_name"]!= "Dribble"].copy()
    events = events.merge(dribble_events[["match_id", "period", "player_name", "x", "y", "Outcome"]].drop_duplicates(
                            subset = ["match_id", "period", "player_name", "x", "y"]), 
                                            on = ["match_id", "period", "player_name", "x", "y"], 
                                            how = "left", validate = "many_to_one", suffixes = ("", "_related_dribble"))
    dribble_events["end_x"] = dribble_events["x"]
    dribble_events["end_y"] = dribble_events["y"]
    events = events.merge(dribble_events[["match_id", "period", "player_name", "end_x", "end_y", "Outcome"]].drop_duplicates(
                            subset = ["match_id", "period", "player_name", "end_x", "end_y"]), 
                                            on = ["match_id", "period", "player_name", "end_x", "end_y"], 
                                            how = "left", validate = "many_to_one", suffixes = ("", "_related_dribble_alternative"))
    events["Outcome"] = np.where(events["Outcome_related_dribble_alternative"].notna(),events["Outcome_related_dribble_alternative"],
                                np.where(events["Outcome_related_dribble"].notna(), events["Outcome_related_dribble"], 
                                        events["Outcome"]))
    events.drop(columns = ["Outcome_related_dribble", "Outcome_related_dribble_alternative"], inplace=True)
    
    return events, frames

In [3]:
def get_outcome_from_events(events):
    # We heuristically calculate the power for the carries for which statsbomb does not provide information.
    events = events.sort_values(["match_id", "period",  "minute", "second"])

    rel_events = events[~events["type_name"].isin(["Pressure", "Duel"])].copy()
    rel_events["gametime_in_seconds"] = events["minute"] * 60 + events["second"]

    shifted_events = rel_events.shift(-1)
    next_match_same_match = rel_events["match_id"] == shifted_events["match_id"]
    next_player_same_player = rel_events["player_name"] ==shifted_events["player_name"]
    next_team_same_team = rel_events["team_name"] ==shifted_events["team_name"]

    success_events = ['Pass','Goal','Dribble','Shot', "Carry", "Ball Receipt", "Foul Won"]
    next_success = shifted_events["type_name"].isin(success_events)

    unsuccessful_events = ["Miscontrol", "Dispossessed", "Interception"]
    next_lost_ball = shifted_events["type_name"].isin(unsuccessful_events)

    ball_recovery_by_opponent = ["Ball Recovery", "Block", "Collected"]
    next_ball_recovery_event = shifted_events["type_name"].isin(ball_recovery_by_opponent)

    foul_by_opponent_or_dribbled_past = ["Foul Committed", "Dribbled Past"]
    next_foul_by_opponent_or_dribbled_past = shifted_events["type_name"].isin(foul_by_opponent_or_dribbled_past)

    successful_outcomes = ["Won", "Success In Play", "Claim", "Complete", "Saved", "Saved Twice", "Success To Team", "Goal"]
    unsuccessful_outcomes = ["Incomplete", "Lost", "Lost Out", "Lost In Play", "Off T", "Out", "Pass Offside"]

    event_more_than_ten_seconds_away = shifted_events["gametime_in_seconds"] - rel_events["gametime_in_seconds"]   > 10

    rel_events["Outcome"] = np.where(
        rel_events["outcome_name"].isin(successful_outcomes), 1, 
        np.where(rel_events["outcome_name"].isin(unsuccessful_outcomes), 0, 
                 np.where(next_match_same_match & next_team_same_team & next_success, 1,
                          np.where(next_match_same_match & next_player_same_player & next_lost_ball ,0,
                                   np.where(next_match_same_match & (~next_team_same_team) & next_success, 0,
                                            np.where(next_match_same_match & (~next_team_same_team) & next_ball_recovery_event, 0,
                                                     np.where(next_match_same_match & (~next_team_same_team) & next_foul_by_opponent_or_dribbled_past, 1,
                                                     2)))))))

    events = events.merge(rel_events[["match_id", "id", "Outcome"]], how = "left", on = ["match_id", "id"], 
                         validate ="one_to_one")
    return events

In [4]:
events, frames = get_event_data(only_female = True)
events = events.fillna(0)
events.head()

509


100%|████████████████████████████████████████████████████████████████████████████████| 509/509 [16:18<00:00,  1.92s/it]


,index,match_id,id,period,minute,second,type_name,sub_type_name,player_name,x,y,end_x,end_y,outcome_name,team_name,under_pressure,position_name,Outcome
0,0,7298,0b483cd2-1d36-49a0-85c2-149a9de553df,1,0,0,Starting XI,0,0,0.0,0.0,0.0,0.0,0,Manchester City WFC,0.0,0,2.0
1,1,7298,2c873afb-73bb-48a1-9fd5-bfe1400b4a83,1,0,0,Starting XI,0,0,0.0,0.0,0.0,0.0,0,Chelsea FCW,0.0,0,2.0
2,2,7298,040940a1-5972-431e-b6ac-e723edd8e7c2,1,0,0,Half Start,0,0,0.0,0.0,0.0,0.0,0,Manchester City WFC,0.0,0,2.0
3,3,7298,5ba286bd-c397-4ac4-b12f-6bace943afce,1,0,0,Half Start,0,0,0.0,0.0,0.0,0.0,0,Chelsea FCW,0.0,0,0.0
4,4,7298,5fa222d3-240a-4b1e-8f21-452f42bbb85c,1,0,0,Ball Receipt,0,Julia Spetsmark,20.0,35.0,0.0,0.0,0,Manchester City WFC,0.0,Left Wing,0.0


In [5]:
print(events['under_pressure'].mean())
print(events[events['type_name'] == 'Carry']['under_pressure'].mean())
print(events[events['type_name'] == 'Shot']['under_pressure'].mean())
print(events[events['type_name'] == 'Pass']['under_pressure'].mean())

0.2219264744184235
0.3831504474333157
0.22597977243994943
0.18480403954946478


In [6]:
w = 30
l = 20
df = events.copy()
df['xbin'] = pd.cut(df.x, w, labels=False)
df['ybin'] = pd.cut(df.y, l, labels=False)
df['xbin_to'] = pd.cut(df.end_x, w, labels=False)
df['ybin_to'] = pd.cut(df.end_y, l, labels=False)

In [7]:
def get_matrix(values, xbins, ybins):
    indices = [(x,y) for x in range(xbins) for y in range(ybins)]
    x_cuts = values['xbin']
    y_cuts = values['ybin']
    bins = values.groupby([x_cuts, y_cuts]).size()
    return bins.reindex(indices, fill_value=0).unstack().fillna(0)

def get_transition_matrix(values, xbins, ybins): 
    indices = [(x1,y1,x2,y2) for x1 in range(xbins) for y1 in range(ybins) for x2 in range(xbins) for y2 in range(ybins)]
    x_from = values['xbin']
    y_from = values['ybin']
    x_to = values['xbin_to']
    y_to = values['ybin_to']
    valid_occurences = values[values['Outcome'] == 1].copy()
    occurence = values.groupby([x_from, y_from, x_to, y_to]).size()
    successful = valid_occurences.groupby([x_from, y_from, x_to, y_to]).size()
    occurence_2d = occurence.reindex(indices).unstack([2,3], fill_value=0).fillna(0)
    successful_2d = successful.reindex(indices).unstack([2,3], fill_value=0).fillna(0)
    return successful_2d.div(occurence_2d.sum(axis=1), axis=0)


In [8]:
def get_relevant_events(df, Pressure = None):
    print(len(df))
    print('Pressure is {}'.format(Pressure))
    if Pressure is not None:
        df = df[df['under_pressure'] == int(Pressure)]
    print(len(df))    
    shots_df = get_matrix(df[df['type_name']=='Shot'], w, l)
    goals_df = get_matrix(df[(df['type_name']=='Shot') & (df['outcome_name'] == 'Goal')], w, l)
    moves_df = get_matrix(df[df['type_name'].isin(['Pass', 'Carry', 'Dribble'])], w, l)
    total_df = moves_df + shots_df

    moves = moves_df / total_df
    shots = shots_df / total_df
    scores = (goals_df / shots_df).fillna(0)
    relevant_events = df[(df['type_name'].isin(['Pass', 'Carry', 'Dribble'])) & (df['Outcome'].isin([0,1]))]
    T = get_transition_matrix(relevant_events, w, l)
    
    return shots, scores, moves, T
    
    

In [9]:
def get_distance(c1,c2):
    # get mid point of the grid
    rx = 120/w*c1 + 120/w/2
    ry = 80/l*c2 + 80/l/2
    # rx = 105 * x # actual x
    # ry = 68 * y # actual y
    gc = {'x': 120, 'y': 40} # goal center
    distance = math.sqrt(math.pow(gc['x']-rx,2) + math.pow(gc['y']-ry,2))
    return distance

def get_angle(c1,c2):
    # get mid point of the grid
    rx = 120/w*c1 + 120/w/2
    ry = 80/l*c2 + 80/l/2
    # rx = 105 * x # actual x
    # ry = 68 * y # actual y
    post1 = {'x': 120, 'y': (80-7.32)/2}
    post2 = {'x': 120, 'y': 80-(80-7.32)/2}
    angle = abs(math.degrees(math.atan2(post1['y']-ry, post1['x']-rx) - math.atan2(post2['y']-ry, post2['x']-rx)))
    return angle

In [10]:
def build_sym_incremental_model(outputfileName, df):
    shots, scores, moves, T = get_relevant_events(df = df, Pressure = None)
    model = so.Model(name='xThreatModel_sym_inc', session=None)
    indices = [(x,y) for x in range(w) for y in range(l)]
    xT = model.add_variables(indices, name='xT')
    err = model.add_variables(indices, name='error')
    err_abs = model.add_variables(indices, name='error_abs', lb=0)
    model.add_constraints(
        (xT[x,y] + err[x,y] == shots.loc[x,y] * scores.loc[x,y] + moves.loc[x,y] * so.expr_sum(T.loc[(x,y),(z,w)] * xT[z,w] for (z,w) in indices) for (x,y) in indices), name='relation')
    model.add_constraints(
        (err_abs[x,y] >= err[x,y] for (x,y) in indices), name='abs_values1')
    model.add_constraints(
        (err_abs[x,y] >= -err[x,y] for (x,y) in indices), name='abs_values2')
    model.add_constraints(
        (xT[x,y] == xT[x, l-y-1] for (x,y) in indices), name='symm_con')
    model.add_constraint(so.expr_sum(err[x,y] for (x,y) in indices) == 0, name='zero_error_total')
    model.add_constraints(
        (xT[x,y] >= xT[z, w] for (x,y) in indices for (z,w) in indices if get_distance(x,y) < get_distance(z,w) and get_angle(x,y) > get_angle(z,w)), name='better_grid')
    sum_err_abs = so.expr_sum(err_abs[x,y] for (x,y) in indices)
    model.set_objective(sum_err_abs, name='total_error', sense='N')
    model.export_mps(filename = outputfileName)

In [11]:
def build_sym_incremental_model_Pressure(outputfileName, df, xT_general, Pressure = True):
    shots, scores, moves, T = get_relevant_events(df, Pressure = Pressure)
    model = so.Model(name='xThreatModel_sym_inc', session=None)
    indices = [(x,y) for x in range(w) for y in range(l)]
    xT = model.add_variables(indices, name='xT')
    err = model.add_variables(indices, name='error')
    err_abs = model.add_variables(indices, name='error_abs', lb=0)
    model.add_constraints(
        (xT[x,y] + err[x,y] == shots.loc[x,y] * scores.loc[x,y] + moves.loc[x,y] * so.expr_sum(T.loc[(x,y),(z,w)] * xT_general.loc[z,w] for (z,w) in indices) for (x,y) in indices), name='relation')
    model.add_constraints(
        (err_abs[x,y] >= err[x,y] for (x,y) in indices), name='abs_values1')
    model.add_constraints(
        (err_abs[x,y] >= -err[x,y] for (x,y) in indices), name='abs_values2')
    model.add_constraints(
        (xT[x,y] == xT[x, l-y-1] for (x,y) in indices), name='symm_con')
    model.add_constraint(so.expr_sum(err[x,y] for (x,y) in indices) == 0, name='zero_error_total')
    model.add_constraints(
        (xT[x,y] >= xT[z, w] for (x,y) in indices for (z,w) in indices if get_distance(x,y) < get_distance(z,w) and get_angle(x,y) > get_angle(z,w)), name='better_grid')
    sum_err_abs = so.expr_sum(err_abs[x,y] for (x,y) in indices)
    model.set_objective(sum_err_abs, name='total_error', sense='N')
    model.export_mps(filename = outputfileName)

In [12]:
def solve_model(inputModel):
    model = Model(solver_name = 'CBC')
    model.read(inputModel)
    print('model has {} vars, {} constraints and {} nzs'.format(model.num_cols, model.num_rows, model.num_nz))
    model.optimize()
    print('number Solutions is {}'.format(model.num_solutions))
    return model

In [48]:
def get_xT_values(df): 
    model_name = 'export_distance.mps'
    #Build model
    build_sym_incremental_model(model_name, df)
    #Solve Model
    model = solve_model(model_name)

    xT_values_list = list()
    for i, v in enumerate(model.vars):
        xT_values_list.append((model.vars[i].x, v))
    xT_values = pd.DataFrame(xT_values_list, columns = ["value", "variable"])
    xT_values.set_index("variable", inplace=True)
    xT_values = xT_values["value"]
            
    xT_values.index = xT_values.index.map(str)
    xT_values = xT_values[xT_values.index.str.startswith('xT')]
    xT_values.index = xT_values.index.str.replace('xT','')
    xT_index = []
    for index in xT_values.index:
        xT_index.append(tuple(map(int, index[1:-1].split(','))))
    xT_values.index = xT_index
    xT_values.index = pd.MultiIndex.from_tuples(xT_values.index)    
    return xT_values.unstack().transpose()

In [49]:
def get_xT_values_Pressure(df, xT_general, Pressure = True): 
    model_name = 'export_distance.mps'
    #Build model
    print("going to build")
    build_sym_incremental_model_Pressure(model_name, df, xT_general, Pressure)
    #Solve Model
    print("going to solve")

    model = solve_model(model_name)
    print("going to rest")
    xT_values_list = list()
    
    for i, v in enumerate(model.vars):
        xT_values_list.append((model.vars[i].x, v))
    xT_values = pd.DataFrame(xT_values_list, columns = ["value", "variable"])
    xT_values.set_index("variable", inplace=True)
    xT_values = xT_values["value"]
            
    xT_values.index = xT_values.index.map(str)
    xT_values = xT_values[xT_values.index.str.startswith('xT')]
    xT_values.index = xT_values.index.str.replace('xT','')
    xT_index = []
    for index in xT_values.index:
        xT_index.append(tuple(map(int, index[1:-1].split(','))))
    xT_values.index = xT_index
    xT_values.index = pd.MultiIndex.from_tuples(xT_values.index)    
    return xT_values.unstack().transpose()

In [52]:
xT_distance_model_general = get_xT_values(df)
xT_distance_model_general

1643292
Pressure is None
1643292
NOTE: Initialized model xThreatModel_sym_inc.
model has 1800 vars, 155005 constraints and 400530 nzs
number Solutions is 1


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.000442,0.000506,0.000623,0.000902,0.000902,0.001071,0.001126,0.001259,0.001430,0.001562,...,0.005777,0.006501,0.007311,0.008012,0.008975,0.009145,0.010301,0.010647,0.008684,0.010043
1,0.000491,0.000623,0.000823,0.000902,0.001071,0.001126,0.001259,0.001430,0.001562,0.001771,...,0.006441,0.007457,0.008696,0.009060,0.010301,0.011309,0.012560,0.012379,0.010114,0.010043
2,0.000494,0.000682,0.000902,0.001002,0.001102,0.001259,0.001430,0.001562,0.001760,0.002006,...,0.006970,0.008012,0.009289,0.010301,0.011339,0.012560,0.013505,0.013716,0.015519,0.013036
3,0.000588,0.000749,0.000959,0.001071,0.001196,0.001385,0.001562,0.001718,0.001878,0.002084,...,0.008012,0.009060,0.010301,0.011339,0.012560,0.014059,0.015860,0.015519,0.019011,0.017286
4,0.000603,0.000902,0.001002,0.001102,0.001259,0.001430,0.001595,0.001771,0.001956,0.002239,...,0.008870,0.009930,0.011339,0.012560,0.015842,0.015860,0.019011,0.021782,0.021950,0.019344
5,0.000714,0.000902,0.001071,0.001196,0.001385,0.001562,0.001698,0.001878,0.002061,0.002239,...,0.009289,0.010934,0.012585,0.015590,0.019657,0.020012,0.022511,0.021950,0.021950,0.021661
6,0.000902,0.001002,0.001102,0.001259,0.001430,0.001562,0.001771,0.001956,0.002113,0.002265,...,0.010044,0.011339,0.014653,0.018030,0.025674,0.028325,0.035914,0.031560,0.038057,0.031922
7,0.000902,0.001002,0.001102,0.001259,0.001430,0.001655,0.001778,0.001974,0.002113,0.002303,...,0.010301,0.013818,0.015842,0.022966,0.028076,0.043321,0.056179,0.084231,0.088080,0.067613
8,0.000902,0.001071,0.001126,0.001259,0.001562,0.001655,0.001791,0.001974,0.002113,0.002303,...,0.011041,0.014829,0.015842,0.023630,0.038794,0.063027,0.067448,0.086182,0.154332,0.198880
9,0.000902,0.001071,0.001126,0.001259,0.001562,0.001655,0.001807,0.001974,0.002141,0.002399,...,0.011591,0.014829,0.015842,0.023630,0.038794,0.063027,0.078880,0.113514,0.250508,0.469376


In [53]:
xT_distance_model_no_pressure = get_xT_values_Pressure(df, xT_general = xT_distance_model_general.T, Pressure = False)
xT_distance_model_no_pressure

going to build
1643292
Pressure is False
1278602
NOTE: Initialized model xThreatModel_sym_inc.
going to solve
model has 1800 vars, 155005 constraints and 310608 nzs
number Solutions is 1
going to rest


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.000334,0.000514,0.000628,0.000795,0.000938,0.001118,0.001168,0.001396,0.001490,0.001616,...,0.005980,0.006492,0.007583,0.008117,0.009106,0.009738,0.011028,0.011110,0.009287,0.012007
1,0.000514,0.000596,0.000741,0.000938,0.001118,0.001168,0.001396,0.001490,0.001616,0.001822,...,0.006544,0.007583,0.008712,0.009106,0.011028,0.012007,0.012990,0.012857,0.012990,0.012007
2,0.000549,0.000696,0.000938,0.001054,0.001134,0.001307,0.001473,0.001616,0.001744,0.001923,...,0.007144,0.008462,0.009428,0.011028,0.012007,0.012990,0.014610,0.016703,0.016703,0.014760
3,0.000628,0.000795,0.000974,0.001118,0.001243,0.001417,0.001616,0.001744,0.001892,0.002081,...,0.008278,0.009106,0.011028,0.012453,0.013156,0.014610,0.017476,0.017829,0.020403,0.017724
4,0.000628,0.000938,0.001054,0.001168,0.001396,0.001490,0.001643,0.001822,0.001987,0.002294,...,0.008712,0.009863,0.012007,0.012990,0.016703,0.018737,0.020403,0.023410,0.021707,0.021572
5,0.000741,0.000938,0.001118,0.001243,0.001417,0.001616,0.001726,0.001923,0.002141,0.002294,...,0.009428,0.011337,0.012990,0.016703,0.021198,0.021198,0.023410,0.023410,0.022619,0.021989
6,0.000795,0.001028,0.001134,0.001307,0.001473,0.001616,0.001822,0.001987,0.002160,0.002357,...,0.009738,0.012007,0.013660,0.016703,0.027655,0.030771,0.036397,0.035892,0.038487,0.033436
7,0.000938,0.001054,0.001134,0.001396,0.001490,0.001686,0.001822,0.002004,0.002160,0.002357,...,0.011028,0.013660,0.015009,0.021968,0.030509,0.030771,0.057409,0.093367,0.091589,0.062708
8,0.000938,0.001118,0.001168,0.001396,0.001616,0.001686,0.001822,0.002004,0.002160,0.002357,...,0.011110,0.014457,0.016703,0.021968,0.030771,0.048478,0.073535,0.116201,0.167787,0.234979
9,0.000938,0.001118,0.001168,0.001396,0.001616,0.001698,0.001836,0.002004,0.002240,0.002413,...,0.012007,0.014457,0.016703,0.021968,0.043206,0.064990,0.090899,0.150734,0.292167,0.504200


In [54]:
xT_distance_model_pressure = get_xT_values_Pressure(df, xT_general = xT_distance_model_general.T, Pressure = True)
xT_distance_model_pressure

going to build
1643292
Pressure is True
364690
NOTE: Initialized model xThreatModel_sym_inc.
going to solve
model has 1800 vars, 155005 constraints and 310608 nzs
number Solutions is 1
going to rest


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.000332,0.000498,0.000602,0.000777,0.000841,0.000974,0.001085,0.001205,0.001286,0.001490,...,0.005507,0.006295,0.006726,0.007484,0.008536,0.008221,0.009599,0.007735,0.007694,0.006438
1,0.000429,0.000509,0.000684,0.000777,0.000974,0.001085,0.001192,0.001286,0.001439,0.001645,...,0.006247,0.006797,0.007637,0.008536,0.009599,0.010051,0.011315,0.011575,0.010602,0.009513
2,0.000429,0.000583,0.000777,0.000958,0.000973,0.001192,0.001286,0.001439,0.001608,0.001784,...,0.006589,0.007484,0.008540,0.009599,0.010051,0.011575,0.011670,0.013409,0.010602,0.010343
3,0.000516,0.000667,0.000841,0.000974,0.001093,0.001286,0.001439,0.001585,0.001753,0.001914,...,0.007715,0.008666,0.009599,0.010518,0.011575,0.013403,0.013409,0.014018,0.015437,0.018509
4,0.000583,0.000777,0.000958,0.001085,0.001192,0.001286,0.001439,0.001753,0.001869,0.002031,...,0.008089,0.009327,0.011014,0.011575,0.014993,0.015116,0.016600,0.019107,0.022431,0.022431
5,0.000662,0.000841,0.000974,0.001093,0.001286,0.001440,0.001608,0.001753,0.001878,0.002117,...,0.008666,0.010110,0.012061,0.013106,0.020628,0.022399,0.020437,0.022431,0.023694,0.022431
6,0.000777,0.000958,0.000974,0.001192,0.001286,0.001439,0.001614,0.001753,0.001987,0.002159,...,0.009735,0.011014,0.014403,0.016528,0.022314,0.022399,0.034320,0.030417,0.037134,0.028369
7,0.000777,0.000958,0.000973,0.001192,0.001286,0.001479,0.001614,0.001835,0.001987,0.002159,...,0.009735,0.012364,0.015734,0.020628,0.023076,0.038345,0.057398,0.069016,0.079082,0.041885
8,0.000777,0.000974,0.001047,0.001192,0.001439,0.001479,0.001662,0.001875,0.001987,0.002281,...,0.010644,0.012364,0.015734,0.023076,0.035508,0.038345,0.057398,0.069016,0.125467,0.099532
9,0.000777,0.000974,0.001085,0.001205,0.001440,0.001479,0.001662,0.001875,0.001987,0.002458,...,0.010644,0.012364,0.015734,0.024001,0.035508,0.046679,0.057398,0.094070,0.200338,0.350718


In [55]:
xT_distance_model_general.to_csv('xT_general_women.csv')
xT_distance_model_pressure.to_csv('xT_pressure_women.csv')
xT_distance_model_no_pressure.to_csv('xT_no_pressure_women.csv')